# **Student Chatbot**

---



# **Intent Classification**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from transformers import BertTokenizer

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Student_Chatbot/quiz_data1.csv")

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['intent'])

In [ ]:

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

label_to_code_mapping = dict(enumerate(train_data['intent'].astype('category').cat.categories))

code_to_label_mapping = {v: k for k, v in label_to_code_mapping.items()}

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_code_mapping))

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = CustomDataset(train_data['text'].tolist(), train_data['intent'].astype('category').cat.codes.tolist(), tokenizer)
val_dataset = CustomDataset(val_data['text'].tolist(), val_data['intent'].astype('category').cat.codes.tolist(), tokenizer)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

# Fine-tune the BERT model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=2e-5)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

epochs = 3
for epoch in range(epochs):

    tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch')

    for batch in tqdm_train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    
        tqdm_train_loader.set_postfix(loss=loss.item())

    # Evaluate on the validation set
    model.eval()
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    all_preds = []
    all_labels = []

    tqdm_val_loader = tqdm(val_loader, desc=f'Validation', unit='batch')

    with torch.no_grad():
        for batch in tqdm_val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Validation Accuracy (Epoch {epoch + 1}): {accuracy * 100:.2f}%')

    model.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation: 100%|██████████| 12/12 [00:00<00:00, 17.46batch/s]


Validation Accuracy (Epoch 1): 80.43%


Validation: 100%|██████████| 12/12 [00:00<00:00, 17.06batch/s]


Validation Accuracy (Epoch 2): 98.91%


Validation: 100%|██████████| 12/12 [00:00<00:00, 17.32batch/s]

Validation Accuracy (Epoch 3): 98.91%


## ***Save the Model***

In [ ]:
# Saving the trained model
model_save_path = 'bert_intent_classification_model.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
# Saving the tokenizer
tokenizer_save_path = 'bert_intent_classification_tokenizer'
tokenizer.save_pretrained(tokenizer_save_path)

('bert_intent_classification_tokenizer/tokenizer_config.json',
 'bert_intent_classification_tokenizer/special_tokens_map.json',
 'bert_intent_classification_tokenizer/vocab.txt',
 'bert_intent_classification_tokenizer/added_tokens.json')

In [ ]:

print(f"Trained model saved at: {model_save_path}")
print(f"Tokenizer saved at: {tokenizer_save_path}")

Trained model saved at: bert_intent_classification_model.pth
Tokenizer saved at: bert_intent_classification_tokenizer


In [ ]:
import shutil


shutil.make_archive(tokenizer_save_path, 'zip', tokenizer_save_path)


print(f"Tokenizer saved at: {tokenizer_save_path}.zip")


Tokenizer saved at: bert_intent_classification_tokenizer.zip


In [ ]:
from google.colab import files


model_save_path = '/content/bert_intent_classification_model.pth'


files.download(model_save_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Testing the model prediction**

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [ ]:

unique_labels_train = train_data['intent'].astype('category').cat.categories
print("Unique Labels in Training Set:", unique_labels_train)

Unique Labels in Training Set: Index(['Feedback Processing', 'Fun Trivia Generation', 'Guidance Provision',
       'Homework Help', 'Quiz Generation'],
      dtype='object')


In [ ]:

loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)


loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_model.pth'))
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_tokenizer')


loaded_model.eval()

# Example usage
text_to_classify = "How can I improve my programming skills?"

inputs = loaded_tokenizer(text_to_classify, return_tensors='pt', truncation=True, padding=True)
outputs = loaded_model(**inputs)
predicted_label = torch.argmax(outputs.logits).item()

print(f"Predicted Label Index: {predicted_label}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Label Index: 2


In [ ]:

print("Label to Code Mapping:", label_to_code_mapping)


predicted_label_index = 3 
print("Predicted Label Index:", predicted_label_index)


if predicted_label_index in label_to_code_mapping:
    predicted_label = label_to_code_mapping[predicted_label_index]
    print(f"Predicted Label: {predicted_label}")
else:
    print(f"Predicted Label Index {predicted_label_index} not found in mapping.")


Label to Code Mapping: {0: 'Feedback Processing', 1: 'Fun Trivia Generation', 2: 'Guidance Provision', 3: 'Homework Help', 4: 'Quiz Generation'}
Predicted Label Index: 3
Predicted Label: Homework Help


In [ ]:
from transformers import BertTokenizer
import torch

loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_tokenizer')
loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_model.pth'))

sample_input = "I want to test myself in science"
sample_input = "I'd like a quiz focused on significant cultural festivals around the world"

inputs = loaded_tokenizer(sample_input, return_tensors='pt', truncation=True, padding=True)

with torch.no_grad():
    outputs = loaded_model(**inputs)

predicted_label = torch.argmax(outputs.logits).item()

print(f"Sample Input: {sample_input}")

print(f"Predicted Label Index: {predicted_label}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sample Input: I'd like a quiz focused on significant cultural festivals around the world
Predicted Label Index: 4


# **Chatbot**

Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers torch pandas
!sudo apt install tesseract-ocr
!pip install pytesseract

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import re
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image
from transformers import pipeline
import requests
import spacy

Set up the Gemini api key

In [ ]:
import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = '...'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

chat_model = genai.GenerativeModel('gemini-pro')
chat = chat_model.start_chat(history=[])

Models

In [ ]:
# Fine tuned model for Intent Classification
intent_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_tokenizer')
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
intent_model.load_state_dict(torch.load('/content/drive/MyDrive/Student_Chatbot/bert_intent_classification_model.pth'))

# Document ques-ans model
qa_pipeline = pipeline(
    "question-answering",
    model="bert-large-uncased-whole-word-masking-finetuned-squad",
    device=0 if torch.cuda.is_available() else -1
)


# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

...

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Chatbot

In [ ]:
db = []
quiz_history = []

In [ ]:
def categorize_input(user_input):

  inputs = intent_tokenizer(user_input, return_tensors='pt', truncation=True, padding=True)

  with torch.no_grad():
    outputs = intent_model(**inputs)

  predicted_label = torch.argmax(outputs.logits).item()

  return predicted_label

In [ ]:
def quiz_generation(user_input, student_name):
  chat = chat_model.start_chat(history=[])
  topic = chat.send_message(f'''Extract only the main topic in which the student wants a quiz
    in this: {user_input}''').text.replace('*', '')

  remark = chat.send_message(f"""Assume that you are a friendly class tutor. {student_name} wants to write
    a quiz on {topic}. Comment on how the topic will be useful for the student in a teacher's
    perspective. Reply infomally in a fun manner.
    No Hi.""").text.replace('*', '')

  print(f'\n{remark}')

  score = 0

  for _ in range(5):
    ques = chat.send_message("Generate 1 MCQ on the topic " + user_input + """. The format for
      generating is
      Question:
      {give the question and options labeled as A-D}
      Answer: [A-D]
      Also return only the correct option as a character[A-D].""").text.replace('*', '')

    question, answer_text = ques.split("Answer:")

    print(f'\n{question}')

    answer = ques[-1]

    print("Choose the correct option")
    user_ans = input("User: ")

    if user_ans == answer:
      score += 1
      print(f'Current Score: {score}')
    else:
      print("Correct Answer: ", end = "")
      print(ques[-1])
      print(f'Points scored: {score}\n')

  fb = f"Scored {score} points in the quiz for {topic}"

  feedback_messages = [
    "Looks like there might have been some confusion. Let's review the material together and try again!",
    "A little improvement is needed, but don't worry, we'll work on it together!",
    "Progress is being made, but let's keep practicing to strengthen understanding.",
    "Good effort! We're on the right track, let's continue working to enhance comprehension.",
    "Great job! You're really grasping the concepts. Let's keep up the momentum!",
    "Fantastic work! You've mastered this material. Keep challenging yourself to maintain this level of understanding!"
  ]

  print(feedback_messages[score])

  feedback_message = f"Scored {score} points in the {topic} quiz."

  feedback_comments = ['Very Poor Performance', 'Poor Improvement', 'Needs Improvement', 'Average Performance', 'Good Performance', 'Fantastic Performance']

  quiz_feedback = f'{feedback_message} {feedback_comments[score]}..'
  db.append(quiz_feedback)

  quiz_history.append(quiz_feedback)


In [ ]:
def read_from_doc():
  try:
    print("\nProvide a valid document link :)")
    print()
    image_url = input("You: ")
    print("\nAsk your question")
    question = input("You: ")
    out = nlp(image_url, question)
    print(out[0]['answer'])
  except:
    print("\nInvalid document path")
    print("\nWould you like to try again? (y/n)")
    print()
    try_again = input("You: ")
    if(try_again == 'y'):
      read_from_doc()

In [ ]:
def homework_help(user, name):
  print("\nDo you have any documents for it? (y/n):")
  have_doc = input("\nYou: ").lower()
  if have_doc == 'y':
    retrieved_ans = read_from_doc()
    if retrieved_ans != None:
      print(f"\nRetrieved Answer: {retrieved_ans}")
      print("\nThis is the answer I could retrieve from the provided document.")

    print("\nDo you need more clarity on this? (y/n)")
    clarity = input("You: ").lower()
    if(clarity == 'n'):
      return

  print("\n Could you please provide the question once more along with any relevant details?")
  ques = input("\nYou: ")
  ans = chat.send_message(f'{name} needs some help in the following: {user}.').text.replace('*', '')
  print(f'\n{ans}')


In [ ]:
def homework_help(user, name):
  print("\nDo you have any documents for it? (y/n):")
  have_doc = input("\nYou: ").lower()
  if have_doc == 'y':
    retrieved_ans = read_from_doc()

    answer = answer_question_from_image(question, image_path)
    print(answer['answer'])

    if retrieved_ans != None:
      print(f"\nRetrieved Answer: {retrieved_ans}")
      print("\nThis is the answer I could retrieve from the provided document.")

    print("\nDo you need more clarity on this? (y/n)")
    clarity = input("You: ").lower()
    if(clarity == 'n'):
      return

  print("\n Could you please provide the question once more along with any relevant details?")
  ques = input("\nYou: ")
  ans = chat.send_message(f'{name} needs some help in the following: {user}.').text.replace('*', '')
  print(f'\n{ans}')

In [ ]:
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def answer_question_from_image(question, image_path):
    text_from_image = extract_text_from_image(image_path)
    answer = qa_pipeline(question=question, context=text_from_image)
    return answer

In [ ]:
def homework_help(user, name):
  print("\nDo you have any documents for it? (y/n):")
  have_doc = input("\nYou: ").lower()
  if have_doc == 'y':
    print("\nProvide a valid document link :)")
    image_path = input("\nYou: ")

    print("\nAsk your question")
    question = input("\nYou: ")

    answer = answer_question_from_image(question, image_path)

    if answer != None:
      print(f"\nRetrieved Answer: {answer['answer']}")
      print("\nThis is the answer I could retrieve from the provided document.")

    print("\nDo you need more clarity on this? (y/n)")
    clarity = input("\nYou: ").lower()
    if(clarity == 'n'):
      return

  print("\nCould you please provide the question once more along with any relevant details?")
  ques = input("\nYou: ")
  ans = chat.send_message(f'{name} needs some help in the following: {user}.').text.replace('*', '')
  print(f'\n{ans}')

In [ ]:
def guidance(user, name):
  print("\nSure!")
  print(f"\n{name}, could you please elaborate further on your response to provide more details?")
  guidance_ques = input("\nYou: ")
  chat = chat_model.start_chat(history=[])
  sol = chat.send_message(f'''{guidance_ques}''').text.replace('*', '')
  print(f"\n{sol}")
  print("\nHope this helped!")

In [ ]:
def get_sentiment(feedback):
    sentiment_result = sentiment_pipeline(feedback)[0]
    label = sentiment_result['label']
    score = sentiment_result['score']
    return label, score

def react_to_feedback(feedback):
    sentiment_label, sentiment_score = get_sentiment(feedback)

    if sentiment_label == 'POSITIVE':
        if sentiment_score >= 0.7:
            return "\nWe're glad you enjoyed it! Thank you for the positive feedback."
        else:
            return "\nThank you for your positive feedback."
    elif sentiment_label == 'NEGATIVE':
        if sentiment_score >= 0.7:
            return "\nWe apologize for the inconvenience. We'll work on improving our service."
        else:
            return "\nThank you for bringing this to our attention. We'll address your concerns."
    else:
        return "\nWe appreciate your feedback."

In [ ]:
def chatbot():
    print("Hi I'm ClassPET, your personalized virtual assistant for all things academic! Whether you need help with homework, study tips, or just someone to chat with during those late-night study sessions, I'm here for you. From tackling tough assignments to providing resources for your next project, I've got your back. Let's navigate through the world of education  together! Just type your questions or concerns, and let's get started on our learning journey!""")
    student_name = input("\nHey there! Before we dive into your questions, may I know your name? It's always nice to address you properly.\n\nYou: ")
    print(f"\nLet's get started {student_name}!")

    while True:
      print("\nFeel free to type anything you'd like to discuss or ask questions about.")
      user_input = input("\nYou: ").lower()

      if(user_input == 'exit'):
        break

      predicted_label = categorize_input(user_input)

      if(predicted_label == 2):
        guidance(user_input, student_name)

      if(predicted_label == 4):
        quiz_generation(user_input, student_name)

      if(predicted_label == 3):
        print("\nSure!!")
        homework_help(user_input, student_name)

      if(predicted_label == 0):
        reaction = react_to_feedback(user_input)
        print(reaction)

if __name__ == "__main__":
    chatbot()

Hi I'm ClassPET, your personalized virtual assistant for all things academic! Whether you need help with homework, study tips, or just someone to chat with during those late-night study sessions, I'm here for you. From tackling tough assignments to providing resources for your next project, I've got your back. Let's navigate through the world of education  together! Just type your questions or concerns, and let's get started on our learning journey!

Hey there! Before we dive into your questions, may I know your name? It's always nice to address you properly.

You: Shreya

Let's get started Shreya!

Feel free to type anything you'd like to discuss or ask questions about.

You: I want a quiz in science

Oh, you want to write a quiz on science, huh? That's awesome! Science is such a fascinating subject, and it's all around us.

You know what they say, "Science is the key to unlocking the secrets of the universe!" And who doesn't want to be a universe-unlocker?

By studying science, you'l